<a href="https://colab.research.google.com/github/Wabinab/ariel_ml_2021/blob/main/Ariel_Space_Challenge_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ariel Mission Space Challenges
By J.W. Chow and R. Cheshire. 

In [2]:
# Code to authorize access to google cloud bucket and mount to drive.  
from google.colab import auth
auth.authenticate_user()

In [3]:
# NOTE: Please press enter once, then move cursor from end of line to somewhere middle, 
# and press enter again, for authentication successfully done. 
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=lhEHFJ6KMdc2HFUfq7OwF8KaPTtnLb&prompt=consent&access_type=offline&code_challenge=P52HV6A-AH12N3eA8PT3_JD56qpDi_BLXedLdKjskXE&code_challenge_method=S256

Enter verification code: 4/1AY0e-g7EUGPNmTQL-MI8Swf-0Peysw8viAgRlrYr00V4hD6e9EomBO3R6sE

You are now logged in as [chowjunwei37@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [5]:
!gcloud config set project sunlit-analyst-309609

Updated property [core/project].
Updated property [core/project].


**Note: There is no confirmation that mounting locally, if modifications made, will modify the file in the bucket.**

In [6]:
# Check where google colab is running so to see if we can prevent network egress charges
# Though I don't think we can stop this charge unless we create buckets in multiple locations
# and mount the correct one. 

# The shared bucket I created is not located in whole US but only South Carolina and Iowa
# because I'm stupid thinking it might save money that way? And I am not sure how to change
# it except for creating a new bucket and transfer the whole bucket from one to another. 
!curl ipinfo.io

{
  "ip": "35.190.150.128",
  "hostname": "128.150.190.35.bc.googleusercontent.com",
  "city": "North Charleston",
  "region": "South Carolina",
  "country": "US",
  "loc": "32.8546,-79.9748",
  "org": "AS15169 Google LLC",
  "postal": "29415",
  "timezone": "America/New_York",
  "readme": "https://ipinfo.io/missingauth"
}

In [7]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!sudo apt-get -y -q update
!sudo apt-get -y -q install gcsfuse
!mkdir -p data
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 arielml_shared data

deb http://packages.cloud.google.com/apt gcsfuse-bionic main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  90607      0 --:--:-- --:--:-- --:--:-- 90607
OK
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 h

In [12]:
import os, os.path

# Assert correct number of files in case of modifications. 

assert len(os.listdir("./data/test_set/noisy_test")) == 53900
assert len(os.listdir("./data/training_set/noisy_train")) == 125600
assert len(os.listdir("./data/training_set/params_train")) == 125600